# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [62]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [63]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df_raw = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df = city_data_df_raw.rename(columns={"Lng":"Lon"})
city_data_df['Max Temp'] = ((city_data_df['Max Temp'] * 9/5) + 32).round(2)
city_data_df.head()

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,-6.25,96,100,4.63,RU,1734142758
1,1,yulinshi,38.2906,109.7494,20.23,48,0,5.94,CN,1734142759
2,2,olonkinbyen,70.9221,-8.7187,29.39,98,100,1.65,SJ,1734142762
3,3,puerto ayora,-0.7393,-90.3518,72.55,98,81,0.89,EC,1734142763
4,4,inhambane,-23.8650,35.3833,75.74,93,40,3.35,MZ,1734142765


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [64]:

# Configure the map plot
city_map_data = city_data_df.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    title = "OSM",
    frame_width = 800,
    frame_height = 800,
    scale = 0.1
)
# Display the map
city_map_data

:Points   [Lon,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [65]:
city_data_df.head()

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,-6.25,96,100,4.63,RU,1734142758
1,1,yulinshi,38.2906,109.7494,20.23,48,0,5.94,CN,1734142759
2,2,olonkinbyen,70.9221,-8.7187,29.39,98,100,1.65,SJ,1734142762
3,3,puerto ayora,-0.7393,-90.3518,72.55,98,81,0.89,EC,1734142763
4,4,inhambane,-23.8650,35.3833,75.74,93,40,3.35,MZ,1734142765


In [66]:
# Narrow down cities that fit criteria and drop any results with null values

city_weather_data_df = city_data_df[(city_data_df['Max Temp'] >= 70) &
                                (city_data_df['Max Temp'] <= 80) &
                                (city_data_df['Humidity'] >= 0) &
                                (city_data_df['Humidity'] <= 50)]

# Drop any rows with null values

city_weather_clean_df = city_weather_data_df.dropna()
# Display sample data

city_weather_clean_df

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,kingston,17.9970,-76.7936,79.75,50,20,2.06,JM,1734142775
31,31,shendi,16.6915,33.4341,71.51,17,1,6.45,SD,1734142817
182,182,niafunke,15.9322,-3.9906,77.76,22,33,4.73,ML,1734143111
225,225,new norfolk,-42.7826,147.0587,70.92,35,99,2.46,AU,1734143190
226,226,gao,16.6362,1.6370,71.49,12,100,5.68,ML,1734143192
243,243,tonota,-21.4833,27.4833,77.47,30,90,2.05,BW,1734143225
349,349,mandiana,10.6333,-8.6833,72.54,27,4,2.69,GN,1734143428
397,397,choma,-16.8065,26.9531,71.91,36,76,2.17,ZM,1734143521
503,503,kinmparana,12.8402,-4.9199,73.80,23,30,3.70,ML,1734143727
558,558,kiffa,16.6166,-11.4045,76.57,18,73,4.11,MR,1734143844


### Step 3: Create a new DataFrame called `hotel_df`.

In [67]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_weather_clean_df[["City", "Country", "Lat", "Lon", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lon,Humidity,Hotel Name
9,kingston,JM,17.9970,-76.7936,50,
31,shendi,SD,16.6915,33.4341,17,
182,niafunke,ML,15.9322,-3.9906,22,
225,new norfolk,AU,-42.7826,147.0587,35,
226,gao,ML,16.6362,1.6370,12,
243,tonota,BW,-21.4833,27.4833,30,
349,mandiana,GN,10.6333,-8.6833,27,
397,choma,ZM,-16.8065,26.9531,36,
503,kinmparana,ML,12.8402,-4.9199,23,
558,kiffa,MR,16.6166,-11.4045,18,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [68]:
# Set parameters to search for a hotel
radius = 10000
params = {"apiKey": geoapify_key,
        "categories": hotel_df,
        "radius": radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row.loc['Lat']
    longitude = row.loc['Lon']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kingston - nearest hotel: No hotel found
shendi - nearest hotel: No hotel found
niafunke - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
gao - nearest hotel: No hotel found
tonota - nearest hotel: No hotel found
mandiana - nearest hotel: No hotel found
choma - nearest hotel: No hotel found
kinmparana - nearest hotel: No hotel found
kiffa - nearest hotel: No hotel found
sinnar - nearest hotel: No hotel found


,City,Country,Lat,Lon,Humidity,Hotel Name
9,kingston,JM,17.9970,-76.7936,50,No hotel found
31,shendi,SD,16.6915,33.4341,17,No hotel found
182,niafunke,ML,15.9322,-3.9906,22,No hotel found
225,new norfolk,AU,-42.7826,147.0587,35,No hotel found
226,gao,ML,16.6362,1.6370,12,No hotel found
243,tonota,BW,-21.4833,27.4833,30,No hotel found
349,mandiana,GN,10.6333,-8.6833,27,No hotel found
397,choma,ZM,-16.8065,26.9531,36,No hotel found
503,kinmparana,ML,12.8402,-4.9199,23,No hotel found
558,kiffa,MR,16.6166,-11.4045,18,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [70]:
# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    scale = 0.1,
    hover_cols=["City", "Hotel Name"]
)
# Display the map
hotel_df_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lon,Lat]   (City,Hotel Name)